In [138]:
import csv
import re
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn import metrics, tree
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, f1_score, precision_score,
                             recall_score)
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB, MultinomialNB

In [139]:
df = pd.read_csv('dataset.tsv', sep='\t', quoting=csv.QUOTE_NONE, dtype=str,
                 header=None, names=["instance", "text", "id", "sentiment", "is_sarcastic"])

In [140]:
text_data = np.array([])
# Read tweets
for text in df.text:
    text_data = np.append(text_data, text)

In [141]:
""" Functions for text pre-processing """


def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", " ", sample)


def remove_punctuation(sample):
    """Remove punctuations from a sample string"""
    punctuations = r'''$!"&'()*+,-./:;<=>?[\]^`{|}~'''
    no_punct = ""
    for char in sample:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct

def myTokenizer(sample):
    """Customized tokenizer"""
    ################################## 1. Remove numbers
    ################################## 2. Remove auspoll thingy
    ################################## 3. Remove starts with au
    new_words = []
    words = sample.split(' ')
    new_words = [word for word in words if len(word) >= 2 and not word.isdigit() and not word.startswith('#aus') and not word.startswith('au')]
    return new_words

def remove_stopwords_NLTK(sample):
    """Remove stopwords using NLTK"""
    stopWords = set(stopwords.words('english'))
    words = myTokenizer(sample)
    filteredText = ""
    for word in words:
        if word not in stopWords:
            filteredText = filteredText + word + " "
    return filteredText.rstrip()


def porter_stem(sample):
    """Stemming"""
    words = myTokenizer(sample)
    ps = PorterStemmer()
    stemmed_text = ""
    for word in words:
        stemmed_text = stemmed_text + ps.stem(word) + " "
    return stemmed_text.rstrip()


def myPreprocessor(sample):
    """Customized preprocessor"""
    sample = remove_URL(sample)
    sample = sample.lower()
    sample = remove_stopwords_NLTK(sample)
    sample = remove_punctuation(sample)
    sample = porter_stem(sample)
    return sample


In [142]:
# try to use sklearn stop_words later
count = CountVectorizer(preprocessor=myPreprocessor, tokenizer=myTokenizer, max_features=800)
bag_of_words = count.fit_transform(text_data)
print(count.get_feature_names())
size = len(count.vocabulary_)
print(len(count.vocabulary_))

['#4corner', '#7new', '#abc730', '#abcnews24', '#afpraid', '#agchatoz', '#alp', '#asylumseek', '#brexit', '#budget2016', '#cfa', '#cfmeu', '#chafta', '#climat', '#climatechang', '#coal', '#corrupt', '#csg', '#csiro', '#csirocut', '#dutton', '#educ', '#election2016', '#environ', '#et', '#faketradi', '#fraudband', '#gonski', '#green', '#greens16', '#humanright', '#icac', '#inequ', '#insid', '#labor', '#laborlaunch', '#latelin', '#leadersdeb', '#liber', '#lnp', '#lnpfail', '#malcolm', '#marriageequ', '#medicar', '#nauru', '#nbn', '#nbnco', '#nbngate', '#negativegear', '#npc', '#panamapap', '#parakeelia', '#peoplesforum', '#putlnplast', '#qanda', '#qldpol', '#refuge', '#renew', '#savemedicar', '#scomo', '#spendomet', '#springst', '#ssm', '#stopstateterror', '#tennew', '#thedrum', '#turnbul', '1st', '2nd', '50%', '50b', '50bn', '@2gbnew', '@abcnew', '@abcnews24', '@albomp', '@australianlabor', '@barnaby_joyc', '@billshortenmp', '@bowenchri', '@cpyne', '@green', '@greghuntmp', '@johndory49',

In [143]:
X = bag_of_words.toarray()
# creating target classes
Y = np.array([])
for text in df.id:
    Y = np.append(Y, text)
# First 1500 for training set, last 500 for test set
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.25, shuffle=False)

In [150]:
clf = MultinomialNB()
model = clf.fit(X_train, y_train)

In [151]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       10000       0.58      0.66      0.62        56
       10001       0.35      0.25      0.29        36
       10002       0.56      0.45      0.50        31
       10003       0.32      0.53      0.40        87
       10004       0.00      0.00      0.00         2
       10005       0.58      0.62      0.60        52
       10006       0.43      0.36      0.40        44
       10007       0.00      0.00      0.00         2
       10008       0.61      0.72      0.66        46
       10009       0.00      0.00      0.00         4
       10010       0.27      0.27      0.27        11
       10011       0.00      0.00      0.00         7
       10012       0.00      0.00      0.00         4
       10013       0.62      0.43      0.51        37
       10014       0.00      0.00      0.00         6
       10015       0.50      0.62      0.56        24
       10016       0.25      0.14      0.18        14
       10017       0.00    

C:\Users\Doodey\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [152]:
y_pred = model.predict(X_train)
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

       10000       0.83      0.84      0.83       188
       10001       0.77      0.64      0.70       104
       10002       0.70      0.81      0.75        99
       10003       0.65      0.76      0.70       271
       10004       1.00      0.40      0.57        15
       10005       0.72      0.79      0.75       142
       10006       0.79      0.81      0.80       145
       10007       0.00      0.00      0.00         5
       10008       0.79      0.84      0.81       117
       10009       0.78      0.58      0.67        12
       10010       0.58      0.62      0.60        45
       10011       0.00      0.00      0.00         6
       10012       0.73      0.52      0.61        21
       10013       0.81      0.88      0.84        67
       10014       1.00      0.30      0.47        23
       10015       0.80      0.97      0.88        95
       10016       0.80      0.44      0.57        45
       10017       0.85    

C:\Users\Doodey\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
